In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification,TrainingArguments,Trainer

In [ ]:
print(AutoModelForSequenceClassification)

<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>


In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
df=pd.read_excel("NewsCategorizer.xlsx")
df

,category,headline,links,short_description,keywords
0,WELLNESS,143 Miles in 35 Days: Lessons Learned,https://www.huffingtonpost.com/entry/running-l...,Resting is part of training. I've confirmed wh...,running-lessons
1,WELLNESS,Talking to Yourself: Crazy or Crazy Helpful?,https://www.huffingtonpost.com/entry/talking-t...,Think of talking to yourself as a tool to coac...,talking-to-yourself-crazy
2,WELLNESS,Crenezumab: Trial Will Gauge Whether Alzheimer...,https://www.huffingtonpost.com/entry/crenezuma...,The clock is ticking for the United States to ...,crenezumab-alzheimers-disease-drug
3,WELLNESS,"Oh, What a Difference She Made",https://www.huffingtonpost.com/entry/meaningfu...,"If you want to be busy, keep trying to be perf...",meaningful-life
4,WELLNESS,Green Superfoods,https://www.huffingtonpost.com/entry/green-sup...,"First, the bad news: Soda bread, corned beef a...",green-superfoods
...,...,...,...,...,...
49995,SPORTS,This Baseball Team Learned There's AÂ Wrong Wa...,https://www.huffingtonpost.com/entry/san-jose-...,Many fans were pissed after seeing the minor l...,san-jose-giants-japanese-heritage-night
49996,SPORTS,Some Young Spurs Fan Dabbed 38 Times In A Sing...,https://www.huffingtonpost.com/entry/dab-kid-s...,"Never change, young man. Never change.",dab-kid-san-antonio-spurs
49997,SPORTS,Rasheed Wallace Ejected From Knicks-Suns Game ...,https://www.huffingtonpost.com/entry/rasheed-w...,Wallace was hit with a first technical for a h...,rasheed-wallace-ejected-knicks-suns-ball-dont-lie
49998,SPORTS,Why Jake Plummer And Other NFL Players Are Pus...,https://www.huffingtonpost.comhttp://extras.de...,They believe CBD could be an alternative to po...,NaN


In [ ]:
df.shape

(50000, 5)

In [ ]:
df.isnull().sum()

,0
category,0
headline,0
links,0
short_description,5
keywords,2706


In [ ]:
df=df.drop(columns={'keywords','links'},axis=1)
df

,category,headline,short_description
0,WELLNESS,143 Miles in 35 Days: Lessons Learned,Resting is part of training. I've confirmed wh...
1,WELLNESS,Talking to Yourself: Crazy or Crazy Helpful?,Think of talking to yourself as a tool to coac...
2,WELLNESS,Crenezumab: Trial Will Gauge Whether Alzheimer...,The clock is ticking for the United States to ...
3,WELLNESS,"Oh, What a Difference She Made","If you want to be busy, keep trying to be perf..."
4,WELLNESS,Green Superfoods,"First, the bad news: Soda bread, corned beef a..."
...,...,...,...
49995,SPORTS,This Baseball Team Learned There's AÂ Wrong Wa...,Many fans were pissed after seeing the minor l...
49996,SPORTS,Some Young Spurs Fan Dabbed 38 Times In A Sing...,"Never change, young man. Never change."
49997,SPORTS,Rasheed Wallace Ejected From Knicks-Suns Game ...,Wallace was hit with a first technical for a h...
49998,SPORTS,Why Jake Plummer And Other NFL Players Are Pus...,They believe CBD could be an alternative to po...


In [ ]:
df=df.dropna()
print(df.isnull().sum())
print("===============")
print(df.info())

category             0
headline             0
short_description    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 49995 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   category           49995 non-null  object
 1   headline           49995 non-null  object
 2   short_description  49995 non-null  object
dtypes: object(3)
memory usage: 1.5+ MB
None


In [ ]:
df=df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45744 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   category           45744 non-null  object
 1   headline           45744 non-null  object
 2   short_description  45744 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [ ]:
category_to_number = {category: i for i, category in enumerate(df['category'].unique())}
df['label'] = df['category'].map(category_to_number)

print(df[['category', 'label']].drop_duplicates().reset_index(drop=True))




         category  label
0        WELLNESS      0
1        POLITICS      1
2   ENTERTAINMENT      2
3          TRAVEL      3
4  STYLE & BEAUTY      4
5       PARENTING      5
6    FOOD & DRINK      6
7      WORLD NEWS      7
8        BUSINESS      8
9          SPORTS      9


<ipython-input-11-733f11c2d960>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['category'].map(category_to_number)


In [ ]:
df['label'].value_counts()

,count
label,
1,5000
2,5000
7,5000
4,5000
5,5000
6,5000
0,4999
3,4996
8,3091


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def get_tokens(batch):
    headline = batch['headline'] if isinstance(batch['headline'], str) else ""
    short_description = batch['short_description'] if isinstance(batch['short_description'], str) else ""

    return tokenizer(headline + " " + short_description, padding='max_length', truncation=True, max_length=128)

In [ ]:
batch_size = 500
tokenized_data = {
    'input_ids': [],
    'attention_mask': [],
    'label': []
}

In [ ]:
for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i+batch_size]
    tokenized_batch = batch_df.apply(get_tokens, axis=1)

    tokenized_data['input_ids'].extend([x['input_ids'] for x in tokenized_batch])
    tokenized_data['attention_mask'].extend([x['attention_mask'] for x in tokenized_batch])
    tokenized_data['label'].extend(batch_df['label'].tolist())


In [ ]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_dict(tokenized_data)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(category_to_number))

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    gradient_accumulation_steps=2,
    fp16=True,
    save_steps=5000,
    logging_steps=1000,
    weight_decay=0.01,
    evaluation_strategy="epoch"  # Evaluate at the end of each epoch
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Tokenize validation data
tokenized_val_df = val_df.apply(get_tokens, axis=1)

# Create validation dataset
eval_dataset = Dataset.from_pandas(pd.DataFrame({
    'input_ids': list(tokenized_val_df.apply(lambda x: x['input_ids'])),
    'attention_mask': list(tokenized_val_df.apply(lambda x: x['attention_mask'])),
    'label': val_df['label']
}))


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Include evaluation dataset
    tokenizer=tokenizer,
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,0.504800,0.302626
2,0.306500,0.169660
3,0.198800,0.097027
4,0.124400,0.051629
5,0.067700,0.032662
6,0.044300,0.017949
7,0.024500,0.012170
8,0.011500,0.005439
9,0.009400,0.004103
10,0.003400,0.003277


TrainOutput(global_step=28590, training_loss=0.13704701114593165, metrics={'train_runtime': 2782.2958, 'train_samples_per_second': 164.411, 'train_steps_per_second': 10.276, 'total_flos': 1.5151132993536e+16, 'train_loss': 0.13704701114593165, 'epoch': 10.0})

In [ ]:
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")

('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/vocab.txt',
 'fine_tuned_model/added_tokens.json',
 'fine_tuned_model/tokenizer.json')

In [ ]:
evaluation_metrics = trainer.evaluate()
# evaluation metrics
print(evaluation_metrics)


{'eval_loss': 0.003277115523815155, 'eval_runtime': 12.8413, 'eval_samples_per_second': 712.468, 'eval_steps_per_second': 89.088, 'epoch': 10.0}


In [ ]:
# Use trainer.evaluate to evaluate on the validation dataset
evaluation_metrics = trainer.evaluate(eval_dataset=eval_dataset)

# Print out the evaluation metrics
print(evaluation_metrics)

# You can also extract the accuracy directly if needed
if 'eval_accuracy' in evaluation_metrics:
    print(f"Validation Accuracy: {evaluation_metrics['eval_accuracy']:.4f}")


{'eval_loss': 0.003277115523815155, 'eval_runtime': 15.2875, 'eval_samples_per_second': 598.464, 'eval_steps_per_second': 74.832, 'epoch': 10.0}


In [ ]:
test_sample = {
    'headline': " 'Barry' takes a whack at its farewell season",
    'short_description':"'Barry' has taken chances from the very beginning, which is certainly true of a fourth and final season that picks up where the third left off, with its hitman-turned-wannabe actor getting arrested."
 } # Tokenize
tokenized_test_sample = tokenizer(
    test_sample['headline'] + " " + test_sample['short_description'],
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors="pt"  # Return as PyTorch tensors
)

# Remove 'token_type_ids' if present
# DistilBERT models typically don't use token_type_ids
if 'token_type_ids' in tokenized_test_sample:
    del tokenized_test_sample['token_type_ids']

# Make sure the input tensors are moved to the same device as the model (GPU if available)
tokenized_test_sample = {key: value.to(model.device) for key, value in tokenized_test_sample.items()}

# Get the model's prediction (logits)
with torch.no_grad():
    outputs = model(**tokenized_test_sample)

# Get the predicted label by taking the argmax of the logits
predicted_label = torch.argmax(outputs.logits, dim=1).item()

# Map the predicted label back to the category name
predicted_category = {v: k for k, v in category_to_number.items()}[predicted_label]

print(f"Predicted category: {predicted_category}")


Predicted category: ENTERTAINMENT


In [ ]:
!zip -r fine_tuned_model.zip fine_tuned_model


  adding: fine_tuned_model/ (stored 0%)
  adding: fine_tuned_model/tokenizer.json (deflated 71%)
  adding: fine_tuned_model/tokenizer_config.json (deflated 76%)
  adding: fine_tuned_model/model.safetensors (deflated 8%)
  adding: fine_tuned_model/config.json (deflated 56%)
  adding: fine_tuned_model/special_tokens_map.json (deflated 42%)
  adding: fine_tuned_model/vocab.txt (deflated 53%)
